In [2]:
import numpy as np

In [3]:
# Function that builds the tree, The structure is a dictionary with entries in the form of "(node,node) : [(distribution, parameters), mgf, edge delay]"
def Build_Tree(file_name):
    file = open(file_name)

    tree = {}

    for line in file:
        dist = line[4:5]
        if dist == 'N':
            mu = int(line[6:7])
            sigma = int(line[8:9])
            tree[(line[:1],line[2:3])] = [(dist,mu,sigma),lambda t:np.exp(mu*t)*np.exp((1/2)*(sigma**2)*(t**2)),0]
        if dist == 'E':
            lam = int(line[6:7])
            tree[(line[:1],line[2:3])] = [(dist,lam),lambda t: lam/(lam-t),0]
        if dist == 'U':
            a = int(line[6:7])
            b = int(line[8:9])
            tree[(line[:1],line[2:3])] = [(dist,a,b),lambda t: (np.exp(t*b)-np.exp(t*a))/(t*(b-a)),0]
        if dist == 'P':
            lam = int(line[6:7])
            tree[(line[:1],line[2:3])] = [(dist,lam),lambda t: np.exp(lam*(np.exp(t)-1)),0]
    return tree

In [4]:
#Simulates the edge delay of a single edge
def Simulate(value):
    dist = value[0][0]
    if dist == 'N':
        value[2] = np.random.normal(value[0][1],value[0][2])
    if dist == 'E':
        value[2] = np.random.exponential(value[0][1])
    if dist == 'U':
        value[2] = np.random.uniform(value[0][1],value[0][2])
    if dist == 'P':
        value[2] = np.random.poisson(value[0][1])

#Uses the distribution and parameters stored in the tree to simulate the edge delays for each edge.
def Simulate_Edges(tree):
    for keys in tree:
        Simulate(tree[keys])

In [8]:
#Creates a tree object that can be more easily handled in the form of a dictionary where every key is a node and every value stored at the key is every adjacent node to the key
def Connection_Tree(tree):
    connection_tree = {}
    for keys in tree:
        if keys[0] not in connection_tree:
            connection_tree[keys[0]] = set(keys[1])
        if keys[1] not in connection_tree:
            connection_tree[keys[1]] = set(keys[0])
        if keys[0] in connection_tree:
            connection_tree[keys[0]].add(keys[1])
        if keys[1] in connection_tree:
            connection_tree[keys[1]].add(keys[0])
    return connection_tree

#Does a recursive DFS on a tree between two given nodes
def DFS(connection_tree, source, observer):
    stack = [(source, [source])]
    visited = set()
    while stack:
        (vertex, path) = stack.pop()
        if vertex not in visited:
            if vertex == observer:
                return path
            visited.add(vertex)
            for neighbor in connection_tree[vertex]:
                stack.append((neighbor,path + [neighbor]))

#Converts a path in the form of consecutive nodes into a path in the form of consecutive edges
def Path_Edge(path):
    edges = []
    for i in range(len(path)-1):
        edges.append((path[i],path[i+1]))
    return edges

#Simulates the infection from a given source node to an observer node
def Infection_Simulation(tree, observers, source):
    infection_times = []
    connection_tree = Connection_Tree(tree)

    #Finds the path each observer to the source using DFS and then adds up the edge costs stored in the tree on those paths
    for observer in observers:
        path = DFS(connection_tree, source, observer)
        edges = Path_Edge(path)
        time = 0
        for key in edges:
            if key in tree:
                time += tree[key][2]
            else:
                flipped_key = (key[1],key[0])
                time += tree[flipped_key][2]
        infection_times.append(time)
    return infection_times

In [58]:
#Finds the joint moment generating function for a set of observer nodes with corresponding times and returns the mgf evaluated at those times.
#Returns a dictionary of the form "source : phi(t|s = source)"
def Joint_MGF(t,tree,observers):
    connection_tree = Connection_Tree(tree)
    sources = {}

    #Finds the list of possible observer nodes
    for node in connection_tree:
        if node not in observers:
            sources[node] = 0

    #Finds the joint mgf for each possible source
    for node in sources:
        edges = []
        mgf = 1

        #creates an list where each element is the path from the source node to each observer
        for i in range(len(observers)):
            path = DFS(connection_tree,node,observers[i])
            edges.append(Path_Edge(path))
        path_eval = {}

        #Creates a dictionary of the form "edge: sum of t's that use that edge"
        for i in range(len(edges)):
            for j in range(len(edges[i])):
                if edges[i][j] not in path_eval and (edges[i][j][1],edges[i][j][0]) not in path_eval:
                    path_eval[edges[i][j]] = t[i]
                else:
                    path_eval[edges[i][j]] += t[i]

        #Evaluates the joint mgf using the dictionary
        for key in path_eval:
            if key in tree:
                phi = tree[key][1](path_eval[key])
            else:
                phi = tree[(key[1],key[0])][1](path_eval[key])
            mgf = mgf * phi
            
        sources[node] = mgf
    return sources

In [57]:
#Some Testing for the joint mgf function
Tree = Build_Tree("test.txt")
Simulate_Edges(Tree)

ta = .4
td = .8

print(Joint_MGF([ta,td],Tree,['a','d']))


print(Tree[('a','b')][1](ta)*Tree[('b','c')][1](td)*Tree[('c','d')][1](td)) #phi(t|s=b)

print(Tree[('a','b')][1](ta)*Tree[('b','c')][1](ta)*Tree[('c','d')][1](td)) #phi(t|s=c)

print(Tree[('a','b')][1](ta)*Tree[('b','c')][1](ta)*Tree[('c','e')][1](ta+td)*Tree[('c','d')][1](td)) #phi(t|s=e)

print(Tree[('a','b')][1](ta)*Tree[('b','c')][1](ta)*Tree[('c','f')][1](ta+td)*Tree[('c','d')][1](td)) #phi(t|s=e)

{'b': 7.817513617382336, 'c': 6.149474015700122, 'e': 11.889582274795437, 'f': 12.633683645845965}
7.817513617382336
6.149474015700122
11.889582274795437
12.633683645845963
